In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Funciones

In [1311]:
def doClean(a):   
    a = str(a)
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', a)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

In [1312]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [1313]:
def union_agrup(df_maestro, df_union, agrup_maestro, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])
        
        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='right')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    print('columnas final: {}'.format(df_union_agrup_match.columns))
        #borramos columnas que sobran
    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    if agrup_maestro != agrup_union:
        df_union_agrup_match.drop(columns=agrup_union, inplace=True)
    print('num registros tras union: {}'.format(df_union_agrup_match.shape))
    
    return df_union_agrup_match

In [1314]:
def union_solo_muni(df_maestro, df_union, muni_maestro, muni_union, threshold=85):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por municipio
    muni_match = fuzzy_merge(df_maestro, df_union, muni_maestro, muni_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(muni_match.shape))
    num_matches = (muni_match[muni_match['matches']!='']['matches'].nunique())
    if (num_matches != muni_match[muni_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        
        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(df_union, muni_match, left_on=muni_union, right_on='matches', how='right')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    df_union_muni_match.drop(columns=['matches'], inplace=True)
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

In [1315]:
def union_muni(df_maestro, df_union, muni_maestro, agrup_maestro, muni_union, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])

        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='right')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    
    #fuzzy por municipio
    df_maestro['muni_agrup'] = df_maestro[muni_maestro]+"/"+df_maestro[agrup_maestro]
    df_union_agrup_match['muni_agrup'] = df_union_agrup_match[muni_union]+"/"+df_union_agrup_match[agrup_maestro]
    print(f'df_union_agrup_match: {df_union_agrup_match.shape}')
   
    muni_match = pd.DataFrame()
    maestro_count = 0
    for agrup in list(maestro_agrup[agrup_maestro]):
        print(agrup)
        maestro_agrup_sel = df_maestro[df_maestro[agrup_maestro] == agrup].copy()
        maestro_count = maestro_agrup_sel.shape[0] + maestro_count
        union_agrup_sel = df_union_agrup_match[df_union_agrup_match[agrup_maestro] == agrup].copy()
        if union_agrup_sel.empty == True:
            print(f'no existe para {agrup}')
        
        else:
            aux = fuzzy_merge(maestro_agrup_sel, union_agrup_sel, 'muni_agrup', 'muni_agrup', threshold=threshold_mun)
            num_matches = (aux[aux['matches']!='']['matches'].nunique())
            if (num_matches != aux[muni_maestro].nunique()):
                print('no se han cruzado todos los valores:')
                #print(aux[aux['matches'].isnull()]) 
            muni_match = pd.concat([muni_match, aux])
    
        #comprobacion registros    
    print('num muni maestro: {}'.format(maestro_count))
    if maestro_count != muni_match.shape[0]:
        print("se han perdido registros")       
        
    muni_match.drop(columns='muni_agrup', inplace=True)
    df_union_agrup_match.drop(columns=[agrup_maestro], inplace=True)

    
    print(f'muni_match: {muni_match.columns}')
    print(f'df_union_agrup_match: {df_union_agrup_match.shape}')

        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(muni_match, df_union_agrup_match, left_on='matches', right_on='muni_agrup', how='left')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar por municipio')        

    #merge maestro con fichero union
    #df_tras_union = pd.merge(df_maestro, df_union_muni_match, left_on=muni_maestro, right_on='matches', how='left')
    
    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    if agrup_maestro != agrup_union:
        df_union_muni_match.drop(columns=agrup_union, inplace=True)
    df_union_muni_match.drop(columns=['matches', 'muni_agrup'], inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

In [1316]:
def union_cod_postal(maestro_nivel_cod_postal_1, df_union_2, df_unido_3, cod_postal_1, cod_postal_2, muni_prov, oper_agrup):
    union_cp = pd.merge(maestro_nivel_cod_postal_1, df_union_2, left_on=cod_postal_1, right_on=cod_postal_2, how='left')
    union_cp.drop(columns=[cod_postal_1, cod_postal_2], inplace=True)
    union_cp = union_cp.groupby(by=muni_prov, as_index=False).agg([oper_agrup]).reset_index()
    union_cp.columns = union_cp.columns.droplevel(1)  
    df = pd.merge(df_unido_3, union_cp, left_on=muni_prov, right_on=muni_prov, how='left')
    return df

In [1317]:
def union_cod_ine(maestro_nivel_ine_1, df_union_2, df_unido_3, ine_code_1, ine_code_2, muni_prov, oper_agrup):
    union_ine = pd.merge(maestro_nivel_ine_1, df_union_2, left_on=ine_code_1, right_on=ine_code_2, how='left')
    union_ine.drop(columns=[ine_code_1, ine_code_2], inplace=True)
    union_ine = union_ine.groupby(by=muni_prov, as_index=False).agg([oper_agrup]).reset_index()
    union_ine.columns = union_ine.columns.droplevel(1)  
    df = pd.merge(df_unido_3, union_ine, left_on=muni_prov, right_on=muni_prov, how='left')
    return df

In [1479]:
def standardize_headers(x):
    x = doClean(x)
    return x

In [1480]:
pattern = re.compile("^[a-zA-Z]+$") 

def doCleanNumber(n):
    n = str(n)
    if pattern.match(n): 
        return 0
    else:
        n = n.strip(' ')
        n = str.replace(n, ",","")
        n = str.replace(n, ".", "")
        if n == "":
            n = str.replace(n, "", "0")
        n = float(n)
        return n

#### Carga y union de ficheros

In [1318]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [1319]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'maestro_nivel_municipio_ori.csv',
 'union_guille2.csv',
 'union_ficheros.ipynb',
 'maestro_nivel_ine.csv',
 'union_mar2.csv',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'maestro_nivel_cod_postal.csv',
 'union_ficheros2.ipynb',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 'union_patri.csv',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'sociedad_sanidad_centros_salud_2019_limpio.csv',
 'maestro_nuevo.csv',
 'union_sandra2.csv',
 'maestro_nivel_municipio.csv',
 'union_ficheros_todos.ipynb',
 'creacion_maestros.ipynb',
 '.ipynb_checkpoints',
 '.~lock.Empresas-SICTED2.xls#',
 'economia_turismo_establecimientos_20XX_limpio.csv',
 'tratamiento',
 'sociedad_usos_suelo_2019_limpio.csv',
 'manual.txt',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'union_patri2.csv',
 'listado-codigos-postales-con-LatyLon.csv',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv',
 'maestro_nivel_todo.csv']

## Fichero maestro

### Creacion maestro nivel municipio

In [1320]:
maestro_nivel_municipio_ori = pd.read_csv("maestro_nivel_municipio_ori.csv", index_col=0)
maestro_nivel_municipio_ori.shape

(6066, 9)

### Creacion maestro nivel cod postal

In [1321]:
maestro_nivel_cod_postal = pd.read_csv("maestro_nivel_cod_postal.csv", index_col=0, dtype={'postal_code':object})

### Creacion maestro nivel ine

In [1322]:
maestro_nivel_ine = pd.read_csv("maestro_nivel_ine.csv", index_col=0, dtype={'ine_code':object})
maestro_nivel_ine.shape

(9511, 2)

## Ficheros especificos

### Ficheros Patri

##### Bibliotecas

In [1323]:
bibliotecas = pd.read_csv("sociedad_cultura_bibliotecas_2017_limipio.csv", index_col=0)
bibliotecas.shape

(3323, 4)

In [1324]:
bibliotecas.head(2)

,municipio_limpio,ccaa,provincia,total_bibliotecas
0,alaior,illes_balears,illes_balears,1
1,alaro,illes_balears,illes_balears,1


In [1325]:
bibliotecas[bibliotecas['municipio_limpio']=='palma']
# idioma municipio = es

,municipio_limpio,ccaa,provincia,total_bibliotecas
38,palma,illes_balears,illes_balears,22


In [1326]:
bibliotecas.rename(columns={'ccaa': 'ccaa_limpio', 'total_bibliotecas': 'num_bibliotecas'}, inplace=True)
bibliotecas.drop(columns=['provincia'], inplace=True)

In [1327]:
# comprobacion duplicados
bibliotecas[bibliotecas.duplicated()]

,municipio_limpio,ccaa_limpio,num_bibliotecas


In [1328]:
union1 = union_muni(maestro_nivel_municipio_ori, bibliotecas, 'cities', 'ccaa', 'municipio_limpio', 'ccaa_limpio', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 3323
num agrupaciones maestro: (19, 1)
num agrupaciones union: (19, 1)
num agrupaciones tras match: (19, 2)
df_union_agrup_match: (3323, 5)
pais_vasco
no se han cruzado todos los valores:
castilla_la_mancha
no se han cruzado todos los valores:
comunitat_valenciana
no se han cruzado todos los valores:
region_de_murcia
no se han cruzado todos los valores:
andalucia
no se han cruzado todos los valores:
castilla_y_leon
no se han cruzado todos los valores:
extremadura
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
cataluna
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
galicia
no se han cruzado todos los valores:
comunidad_de_madrid
no se han cruzado todos los valores:
aragon
no se han cruzado todos los valores:
comunidad_foral_de_navarra
no se han cruzado todos los valores:
principado_de_asturias
no se han c

In [1329]:
union1[union1['cities']=='soneja']

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas
1058,soneja,soneja,castellon,castellon,comunidad_valenciana,comunitat_valenciana,soneja/castellon,Castellón/Castelló,Soneja,1.0


In [1330]:
union1[~union1['num_bibliotecas'].isnull()].shape

(3791, 10)

In [1331]:
union1.shape

(6066, 10)

##### Hospitales

In [1332]:
hospitales = pd.read_csv("sociedad_sanidad_hospitales_2019_limpio.csv", index_col=0, dtype={'cp': object})
hospitales.shape

(613, 2)

In [1333]:
hospitales.head(2)

,cp,uds_hospiatales
0,01002,1
1,01006,1


In [1334]:
maestro_nivel_cod_postal.columns

Index(['postal_code', 'cities_es/province'], dtype='object')

In [1335]:
hospitales.rename(columns={'uds_hospiatales': 'num_hospitales'}, inplace=True)

In [1336]:
union2 = union_cod_postal(maestro_nivel_cod_postal, hospitales, union1, 'postal_code', 'cp', 'cities_es/province', 'sum')
union2.shape

(6066, 11)

In [1337]:
union2.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,num_hospitales
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,0.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,0.0
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1.0,0.0
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,1.0,0.0
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,NaN,0.0


In [1338]:
union2[union2['num_hospitales']>0].shape

(338, 11)

##### Centros salud

In [1339]:
centros_salud = pd.read_csv("sociedad_sanidad_centros_salud_2019_limpio.csv", index_col=0, dtype={'cp': object})
centros_salud.shape

(8316, 2)

In [1340]:
centros_salud.head(2)

,cp,num_centros_salud
0,01001,1
1,01002,2


In [1341]:
union3 = union_cod_postal(maestro_nivel_cod_postal, centros_salud, union2, 'postal_code', 'cp', 'cities_es/province', 'sum')
union3.shape

(6066, 12)

In [1342]:
union3[union3['num_centros_salud']>0].shape

(5880, 12)

##### Colegios

In [1343]:
colegios = pd.read_csv("sociedad_educacion_colegios_2020_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [1344]:
colegios.shape

(7838, 5)

In [1345]:
colegios.head(2)

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
0,00000,a_coruna,a_coruna,6.0,NaN
1,00000,a_pontenova,lugo,NaN,1.0


In [1346]:
colegios[colegios['codigo_postal']=='01450']
#codigos postales erroneos, se une por municipio/provincia y cambiando de idioma

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos


In [1347]:
colegios[colegios['municipio_limpio']=='alegria_dulantzi']
# idioma municipio = es

,codigo_postal,municipio_limpio,provincia,num_colegios_privados,num_colegios_publicos
81,01240,alegria_dulantzi,araba,NaN,3.0


In [1348]:
colegios.drop(columns=['codigo_postal'], inplace=True)
colegios.rename(columns={'provincia': 'provincia_limpio'}, inplace=True)
colegios = colegios.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()
colegios.shape

(6331, 4)

In [1349]:
union4 = union_muni(union3, colegios, 'cities_es', 'province', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 6331
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
df_union_agrup_match: (6331, 6)
araba
no se han cruzado todos los valores:
gipuzkoa
no se han cruzado todos los valores:
bizkaia
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
cuenca
no se han cruzado todos los valores:
guadalajara
no se han cruzado todos los valores:
toledo
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
jaen
no se han cruzado todos

In [1350]:
union4[union4['num_colegios_publicos']>0.0].shape

(4849, 14)

In [1351]:
union4.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province', 'provincia_original', 'municipio_original',
       'num_bibliotecas', 'num_hospitales', 'num_centros_salud',
       'num_colegios_privados', 'num_colegios_publicos'],
      dtype='object')

In [1352]:
union4.shape

(6066, 14)

##### Guardia Civil

In [1353]:
guardia_civil = pd.read_csv("sociedad_seguridad_guardia_civil_2019_limpio.csv", index_col=0, dtype={'cp': object})

In [1354]:
guardia_civil.shape

(2237, 4)

In [1355]:
guardia_civil.head(5)
#codigo postal no valido

,cp,municipio_limpio,provincia,uds_guardia_civil
0,10003,caceres,caceres,1
1,1010,vitoria,alava,1
2,10100,miajadas,caceres,1
3,10110,madrigalejo,caceres,1
4,10120,logrosan,caceres,1


In [1356]:
guardia_civil[guardia_civil['municipio_limpio']=='benicasim']
# idioma municipio = es

,cp,municipio_limpio,provincia,uds_guardia_civil
153,12560,benicasim,castellon,1


In [1357]:
guardia_civil.drop(columns=['cp'], inplace=True)
guardia_civil.rename(columns={'provincia': 'provincia_limpio', 'uds_guardia_civil': 'num_guardia_civil'}, inplace=True)
guardia_civil = guardia_civil.groupby(by=['municipio_limpio','provincia_limpio'], as_index=False).sum()
guardia_civil.shape

(2139, 3)

In [1358]:
union5 = union_muni(union4, guardia_civil, 'cities_es', 'province_es', 'municipio_limpio', 'provincia_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 2139
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
df_union_agrup_match: (2139, 5)
alava
no se han cruzado todos los valores:
guipuzkoa
no se han cruzado todos los valores:
vizcaya
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
cuenca
no se han cruzado todos los valores:
guadalajara
no se han cruzado todos los valores:
toledo
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
cordoba
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
jaen
no se han cruzado todo

In [1359]:
union5[~union5['num_guardia_civil'].isnull()].shape

(4355, 15)

In [1360]:
union5.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,num_hospitales,num_centros_salud,num_colegios_privados,num_colegios_publicos,num_guardia_civil
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,0.0,1.0,0.0,3.0,NaN
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,0.0,1.0,3.0,6.0,NaN


In [1361]:
union5.shape

(6066, 15)

##### Universidades

In [1362]:
universidad = pd.read_csv('sociedad_educacion_universidad_2018_limpio.csv', index_col=0, dtype={'cp': object})

In [1363]:
universidad.shape

(165, 6)

In [1364]:
universidad.head(2)

,municipio_limpio,ccaa,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,albacete,castilla_la_mancha,NaN,1.0,NaN,52.0
1,alcala_de_henares,madrid,1.0,1.0,5.0,114.0


In [1365]:
#### universidad['ccaa'].drop_duplicates().sort_values()

In [1366]:
universidad['ccaa'].replace("balears","illes_balears", inplace=True, regex=True)
universidad['ccaa'].replace("madrid","comunidad_de_madrid", inplace=True, regex=True)
universidad['ccaa'].replace("navarra","comunidad_foral_de_navarra", inplace=True, regex=True)
universidad['ccaa'].replace("asturias","principado_de_asturias", inplace=True, regex=True)
universidad['ccaa'].replace("rioja","la_rioja", inplace=True, regex=True)
universidad['ccaa'].replace("murcia","region_de_murcia", inplace=True, regex=True)

In [1367]:
universidad.rename(columns={'ccaa': 'ccaa_limpio'}, inplace=True)
universidad.shape

(165, 6)

In [1368]:
union6 = union_muni(union5, universidad, 'cities_es', 'ccaa', 'municipio_limpio', 'ccaa_limpio', threshold=85, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 165
num agrupaciones maestro: (19, 1)
num agrupaciones union: (17, 1)
num agrupaciones tras match: (19, 2)
no se han cruzado todos los valores:
         ccaa matches
6064    ceuta        
6065  melilla        
el fichero de union no dispone de todos los valores para cruzar
df_union_agrup_match: (167, 8)
pais_vasco
no se han cruzado todos los valores:
castilla_la_mancha
no se han cruzado todos los valores:
comunitat_valenciana
no se han cruzado todos los valores:
region_de_murcia
no se han cruzado todos los valores:
andalucia
no se han cruzado todos los valores:
castilla_y_leon
no se han cruzado todos los valores:
extremadura
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
cataluna
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
galicia
no se han cruzado todos los valores:
comunidad_de_madrid
no se han cruz

In [1369]:
union6.head(5)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,num_hospitales,num_centros_salud,num_colegios_privados,num_colegios_publicos,num_guardia_civil,num_universidades_privadas,num_universidades_publicas,num_tit_uni_privadas,num_tit_uni_publicas
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,0.0,1.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,0.0,1.0,3.0,6.0,NaN,NaN,NaN,NaN,NaN
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,1.0,0.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1370]:
union6.shape

(6066, 19)

##### Establecimientos

In [1371]:
establecimientos = pd.read_csv("economia_turismo_establecimientos_20XX_limpio.csv", index_col=0, dtype={'codigo_postal': object})

In [1372]:
establecimientos.shape

(2063, 35)

In [1373]:
establecimientos.head(2)

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
0,00000,aldea_de_cuenca_hinojares,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000,arroyo_frio_la_iruela,andalucia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [1374]:
# mismo codigo postal para mas de un municipio, se producen duplicados
establecimientos[establecimientos['codigo_postal']=='01118']

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
18,01118,bernedo,euskadi_pais_vasco,NaN,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,01118,lagran,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,01118,urturi,euskadi_pais_vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1375]:
establecimientos[establecimientos['municipio_limpio']=='alcoi']
# municipio en 2 idiomas, se cruza por cp

,codigo_postal,municipio_limpio,ccaa,agencias_de_viajes,albergues,alojamientos_rurales,artesanos,atraque_de_cruceros_y_ferries,bares_y_cafeterías,bodegas,...,otros_servicios,palacios_de_congresos,playas,puertos_deportivos,restaurantes_y_empresas_turísticas_de_catering,seguridad_ciudadana,servicios_de_limpieza,servicios_turísticos_de_salud,taxis,turismo_industrial
163,03800,alcoi,comunitat_valenciana,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
165,03801,alcoi,comunitat_valenciana,1.0,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
167,03802,alcoi,comunitat_valenciana,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
170,03804,alcoi,comunitat_valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [1376]:
establecimientos.drop(columns=['municipio_limpio','ccaa'], inplace=True)
establecimientos = establecimientos.groupby(['codigo_postal'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else x.head(1))

In [1377]:
new_names = [(i,'num_establec_'+i) for i in establecimientos.iloc[:, 1:].columns.values]
establecimientos.rename(columns = dict(new_names), inplace=True)

In [1378]:
union7 = union_cod_postal(maestro_nivel_cod_postal, establecimientos, union6, 'postal_code', 'codigo_postal', 'cities_es/province', 'sum')
union7.shape

(6066, 51)

In [1379]:
union7[~union7['num_establec_agencias_de_viajes'].isnull()].shape

(6066, 51)

##### Usos suelo

In [1380]:
usos_suelo = pd.read_csv("sociedad_usos_suelo_2019_limpio.csv", index_col=0)

In [1381]:
usos_suelo.shape

(7610, 12)

In [1382]:
usos_suelo.head(2)

,municipio_limpio,provincia,superficie_urbana,uds_almacen,uds_comercial,uds_cultural,uds_hosteleria,uds_industrial,uds_deportivo,uds_oficinas,uds_religioso,uds_espectaculos
0,abengibre,albacete,"302,079",13,3,2,1,169,1,2,1,0
1,alatoz,albacete,"339,890",41,2,1,1,140,4,2,2,0


In [1383]:
#usos_suelo['provincia'].drop_duplicates()
# no esta el pais vasco - se deescarta

In [1384]:
union7.to_csv("union_patri2.csv")

### Ficheros Mar

##### Poblacion por género y edad

In [1385]:
poblacion = pd.read_csv('../mar/pob_municipio_edad_genero.csv',sep = ';') 

In [1386]:
poblacion_genero = poblacion.rename(columns={'cp':'codigo ine','municipio':'localidad'})
poblacion_genero['codigo ine']=poblacion_genero['codigo ine'].apply("{:05d}".format)

In [1387]:
poblacion_genero.drop(['Hombres'], axis=1,  inplace=True)
poblacion_genero.drop(['localidad'], axis=1,  inplace=True)
poblacion_genero = poblacion_genero.rename(columns=standardize_headers)

In [1388]:
poblacion_genero[poblacion_genero.duplicated()].shape

(0, 45)

In [1389]:
union_mar_1 = union_cod_ine(maestro_nivel_ine, poblacion_genero, maestro_nivel_municipio_ori, 'ine_code', 'codigo_ine', 'cities_es/province', 'sum')

In [1390]:
union_mar_1.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,mujeres_de_55_59,mujeres_de_60_64,mujeres_de_65_69,mujeres_de_70_74,mujeres_de_75_79,mujeres_de_80_84,mujeres_de_85_89,mujeres_de_90_94,mujeres_de_95_99,mujeres_de_100_y_mas
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,...,9771.0,9257.0,8470.0,7563.0,7115.0,5125.0,4237.0,3433.0,1724.0,475.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,...,700.0,692.0,556.0,473.0,381.0,294.0,264.0,223.0,128.0,35.0


In [1391]:
union_mar_1.shape

(6066, 54)

##### Parque automovilistico

In [1392]:
parque_automovilistico_initial=pd.read_csv('../mar/parque_automoviles_2018.csv', encoding="utf-8",delim_whitespace=True)
parque_automovilistico_initial.shape

(53, 10)

In [1393]:
parque_automovilistico_initial.head(2)

,PROVINCIAS,CAMIONES,FURGONETAS,AUTOBUSES,TURISMOS,MOTOCICLETAS,TRACTORES_INDUSTRIALES,R_Y_S,OTROS_VEHÍCULOS,TOTAL
0,Araba/Álava,14473,14749,538,158049,16011,1334,3208,4969,213331
1,Albacete,28809,21770,418,191601,25960,3438,5708,4224,281928


In [1394]:
parque_automovilistico_initial.drop_duplicates(subset=['PROVINCIAS'], inplace=True)
parque_automovilistico_initial['PROVINCIAS'] = parque_automovilistico_initial['PROVINCIAS'].apply(doClean)

In [1395]:
parque_automovilistico_initial['PROVINCIAS'].replace("balears_","illes_balears", inplace=True, regex=True)
parque_automovilistico_initial['PROVINCIAS'].replace("rioja_","la_rioja", inplace=True, regex=True)
parque_automovilistico_initial['PROVINCIAS'].replace("palmas_","las_palmas", inplace=True, regex=True)

In [1396]:
parque_automovilistico_initial.rename(columns={'TOTAL': 'total_vehiculos'}, inplace=True)
parque_automovilistico_initial = parque_automovilistico_initial.rename(columns=standardize_headers)

new_names = [(i,'prov_'+i) for i in parque_automovilistico_initial.iloc[:, 1:].columns.values]
parque_automovilistico_initial.rename(columns = dict(new_names), inplace=True)

In [1397]:
union_agrup_mar_2 = union_agrup(maestro_nivel_municipio_ori, parque_automovilistico_initial, 'province', 'provincias', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 53
num agrupaciones maestro: (52, 1)
num agrupaciones union: (53, 1)
num agrupaciones tras match: (52, 2)
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['provincias', 'prov_camiones', 'prov_furgonetas', 'prov_autobuses',
       'prov_turismos', 'prov_motocicletas', 'prov_tractores_industriales',
       'prov_r_y_s', 'prov_otros_vehiculos', 'prov_total_vehiculos',
       'province', 'matches'],
      dtype='object')
num registros tras union: (52, 10)


In [1398]:
union_agrup_mar_2.head(2)

,prov_camiones,prov_furgonetas,prov_autobuses,prov_turismos,prov_motocicletas,prov_tractores_industriales,prov_r_y_s,prov_otros_vehiculos,prov_total_vehiculos,province
0,14473,14749,538,158049,16011,1334,3208,4969,213331,araba
1,28809,21770,418,191601,25960,3438,5708,4224,281928,albacete


In [1399]:
union_mar_2 = pd.merge(union_mar_1, union_agrup_mar_2, left_on='province',right_on='province', how='left')

In [1400]:
# comprobacion duplicados
union_mar_2[union_mar_2.duplicated()]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,mujeres_de_100_y_mas,prov_camiones,prov_furgonetas,prov_autobuses,prov_turismos,prov_motocicletas,prov_tractores_industriales,prov_r_y_s,prov_otros_vehiculos,prov_total_vehiculos


In [1401]:
union_mar_2.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,mujeres_de_100_y_mas,prov_camiones,prov_furgonetas,prov_autobuses,prov_turismos,prov_motocicletas,prov_tractores_industriales,prov_r_y_s,prov_otros_vehiculos,prov_total_vehiculos
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,...,475.0,14473,14749,538,158049,16011,1334,3208,4969,213331
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,...,35.0,14473,14749,538,158049,16011,1334,3208,4969,213331


In [1402]:
union_mar_2.shape

(6066, 63)

##### Latitud longitud y altitud

In [1403]:
altitud_coordenadas_distancias_minimas=pd.read_csv('../mar/altitud_coordenadas_distancias_minimas.csv',index_col=0,dtype={'columns': float})
altitud_coordenadas_distancias_minimas.shape

(8146, 8)

In [1404]:
altitud_coordenadas_distancias_minimas.head(2)

,comunidad,provincia,población,latitud,longitud,altitud,distanica_km,cuidad_mas_cercana
0,andalucia,almeria,abla,37.14114,-2.780104,871.1684,46.876723,El_Ejido
1,andalucia,almeria,abrucena,37.13305,-2.797098,976.9387,45.944207,El_Ejido


In [1405]:
#altitud_coordenadas_distancias_minimas[altitud_coordenadas_distancias_minimas['población']=='mieres']

In [1406]:
altitud_coordenadas_distancias_minimas = altitud_coordenadas_distancias_minimas.sort_values(['provincia','población', 'latitud', 'longitud', 'altitud'])

In [1407]:
#altitud_coordenadas_distancias_minimas[altitud_coordenadas_distancias_minimas.duplicated(subset=['población', 'provincia'])]

In [1408]:
altitud_coordenadas_distancias_minimas.drop_duplicates(subset=['provincia','población'], keep='first', inplace=True)

In [1409]:
altitud_coordenadas_distancias_minimas.drop(columns='comunidad', inplace=True)
altitud_coordenadas_distancias_minimas = altitud_coordenadas_distancias_minimas.rename(columns=standardize_headers)
altitud_coordenadas_distancias_minimas['poblacion'] = altitud_coordenadas_distancias_minimas['poblacion'].apply(doClean)
altitud_coordenadas_distancias_minimas['provincia'] = altitud_coordenadas_distancias_minimas['provincia'].apply(doClean)
altitud_coordenadas_distancias_minimas['cuidad_mas_cercana'] = altitud_coordenadas_distancias_minimas['cuidad_mas_cercana'].apply(doClean)
altitud_coordenadas_distancias_minimas.rename(columns={'distanica_km': 'distanica_km_ciudad_cercana'}, inplace=True)

In [1410]:
union_mar_3 = union_muni(union_mar_2, altitud_coordenadas_distancias_minimas, 'cities_es', 'province_es', 'poblacion', 'provincia', threshold=85, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 8112
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
df_union_agrup_match: (8112, 9)
alava
no se han cruzado todos los valores:
albacete
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
murcia
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
avila
no se han cruzado todos los valores:
badajoz
no se han cruzado todos los valores:
illes_balears
no se han cruzado todos los valores:
barcelona
no se han cruzado todos los valores:
tarragona
no se han cruzado todos los valores:
burgos
no se han cruzado todos los valores:
cantabria
no se han cruzado todos los valores:
la_rioja
no se han cruzado todos los valores:
palencia
no se han cruzado todos los valores:
caceres
no se han cruzado todos los valores:
cadiz
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
ciudad_real
no se han cr

In [1411]:
union_mar_3.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_motocicletas,prov_tractores_industriales,prov_r_y_s,prov_otros_vehiculos,prov_total_vehiculos,latitud,longitud,altitud,distanica_km_ciudad_cercana,cuidad_mas_cercana
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,124813.0,...,16011,1334,3208,4969,213331,42.84149,-2.513507,561.6857,34.443702,bilbao
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,8506.0,...,16011,1334,3208,4969,213331,43.05265,-3.001022,219.6910,34.194237,getxo


In [1412]:
union_mar_3.shape

(6066, 67)

##### Climatologia

In [1413]:
climatologia = pd.read_csv('../mar/climatologia_limpio.csv',index_col=0,dtype={'columns': float})
climatologia.shape

(51, 16)

In [1414]:
#faltan islas baleares
climatologia.head(2)

,n_llu_anual,n_nub_anual,n_nie_anual,n_gra_anual,n_fog_anual,n_des_anual,n_cub_anual,n_tor_anual,nt_00_anual,nt_30_anual,nw_55_anual,nw_91_anual,w_med_anual,ta_max_anual,ta_min_anual,PROVINCIA
0,353.5,305.5,0.0,12.5,151.5,93.0,149.0,11.5,4.772727,5.272727,26.900000,0.700,25.681818,33.5(31/may),3.3(04/ene),coruna
1,76.5,104.0,0.0,2.0,14.0,62.5,16.0,11.0,45.500000,86.500000,2.477273,0.125,10.071429,38.7(12/jul),-8.6(11/ene),albacete


In [1415]:
new_names = [(i,'prov_'+i) for i in climatologia.iloc[:, 0:15].columns.values]
climatologia.rename(columns = dict(new_names), inplace=True)

In [1416]:
union_agrup_mar_4 = union_agrup(maestro_nivel_municipio_ori, climatologia, 'province', 'PROVINCIA', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 51
num agrupaciones maestro: (52, 1)
num agrupaciones union: (51, 1)
num agrupaciones tras match: (52, 2)
no se han cruzado todos los valores:
          province matches
634  illes_balears        
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['prov_n_llu_anual', 'prov_n_nub_anual', 'prov_n_nie_anual',
       'prov_n_gra_anual', 'prov_n_fog_anual', 'prov_n_des_anual',
       'prov_n_cub_anual', 'prov_n_tor_anual', 'prov_nt_00_anual',
       'prov_nt_30_anual', 'prov_nw_55_anual', 'prov_nw_91_anual',
       'prov_w_med_anual', 'prov_ta_max_anual', 'prov_ta_min_anual',
       'PROVINCIA', 'province', 'matches'],
      dtype='object')
num registros tras union: (52, 16)


In [1417]:
union_agrup_mar_4.head(2)

,prov_n_llu_anual,prov_n_nub_anual,prov_n_nie_anual,prov_n_gra_anual,prov_n_fog_anual,prov_n_des_anual,prov_n_cub_anual,prov_n_tor_anual,prov_nt_00_anual,prov_nt_30_anual,prov_nw_55_anual,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual,province
0,353.5,305.5,0.0,12.5,151.5,93.0,149.0,11.5,4.772727,5.272727,26.900000,0.700,25.681818,33.5(31/may),3.3(04/ene),coruna
1,76.5,104.0,0.0,2.0,14.0,62.5,16.0,11.0,45.500000,86.500000,2.477273,0.125,10.071429,38.7(12/jul),-8.6(11/ene),albacete


In [1418]:
union_mar_4 = pd.merge(union_mar_3, union_agrup_mar_4, left_on='province',right_on='province', how='left')

In [1419]:
# comprobacion duplicados
union_mar_4[union_mar_4.duplicated()]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_n_des_anual,prov_n_cub_anual,prov_n_tor_anual,prov_nt_00_anual,prov_nt_30_anual,prov_nw_55_anual,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual


In [1420]:
union_mar_4.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_n_des_anual,prov_n_cub_anual,prov_n_tor_anual,prov_nt_00_anual,prov_nt_30_anual,prov_nw_55_anual,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,124813.0,...,27.0,120.0,19.0,48.0,34.0,3.5,0.25,12.545455,39.7(29/jun),-7.7(06/ene)
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,8506.0,...,27.0,120.0,19.0,48.0,34.0,3.5,0.25,12.545455,39.7(29/jun),-7.7(06/ene)


In [1421]:
union_mar_4.shape

(6066, 82)

##### contaminantes

In [1422]:
contaminantes = pd.read_csv('../mar/contaminantes_horario.csv',index_col=0,dtype={'columns': float})
contaminantes.shape

(65536, 10)

In [1423]:
contaminantes.head(2)

,city,country,utc,local,parameter,value,unit,latitude,longitude,attribution
location,,,,,,,,,,
ES2068A,Navarra,ES,2020-02-29T23:00:00.000Z,2020-03-01T00:00:00+01:00,no2,2.0,µg/m³,43.081900,-1.912700,"[{""name"":""EEA"",""url"":""http://www.eea.europa.eu..."
ES1684A,Barcelona,ES,2020-02-29T23:00:00.000Z,2020-03-01T00:00:00+01:00,no2,37.0,µg/m³,41.492083,2.042497,"[{""name"":""EEA"",""url"":""http://www.eea.europa.eu..."


In [1424]:
ciudades=contaminantes.city.unique()

fill_rows=[]
for i in range(len(ciudades)) : 
    fill_rows.append(0)

indice =[]
contaminantes_ES_feb2020 = pd.DataFrame({'ciudades': ciudades ,'co':fill_rows,'no2':fill_rows,'o3':fill_rows,'pm10':fill_rows,'pm25':fill_rows,'so2':fill_rows})


In [1425]:
indice =[]
contaminantes_ES_feb2020 = pd.DataFrame(index = indice , columns = ['co','no2','o3','pm10','pm25','so2'])
save_city=[]

for ciudad in ciudades:
    #print(ciudad)
    
    for num_parameters in contaminantes[(contaminantes['city'] == ciudad)].groupby('parameter').mean().index :
        
        #print(num_parameters)
        
        contaminantes_ES_feb2020.loc[ciudad,num_parameters]=contaminantes[(contaminantes['city'] == ciudad)].groupby('parameter').mean().loc[num_parameters,'value']
    
    

In [1426]:
contaminantes_ES_feb2020_limpio=contaminantes_ES_feb2020.reset_index()
contaminantes_ES_feb2020_limpio=contaminantes_ES_feb2020_limpio.rename(columns = {'index':'ciudad'})
contaminantes_ES_feb2020_limpio['ciudad']=contaminantes_ES_feb2020_limpio['ciudad'].apply(doClean)

In [1427]:
contaminantes_ES_feb2020_limpio.head()

,ciudad,co,no2,o3,pm10,pm25,so2
0,navarra,349.716,15.0428,51.0958,44.9275,NaN,3.02055
1,barcelona,275.38,25.9469,44.1475,43.7906,NaN,1.73006
2,araba,181.641,11.2318,60.0819,57.1706,17.9722,1.87143
3,zaragoza,213.015,26.6824,36.3669,31.1398,NaN,4.4432
4,illes_balears,398.276,8.99995,91.9815,55.849,10.4833,2.30633


In [1428]:
new_names = [(i,'prov_'+i) for i in contaminantes_ES_feb2020_limpio.iloc[:, 1:].columns.values]
contaminantes_ES_feb2020_limpio.rename(columns = dict(new_names), inplace=True)

In [1429]:
union_agrup_mar_5 = union_agrup(maestro_nivel_municipio_ori, contaminantes_ES_feb2020_limpio, 'province', 'ciudad', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 50
num agrupaciones maestro: (52, 1)
num agrupaciones union: (50, 1)
num agrupaciones tras match: (52, 2)
no se han cruzado todos los valores:
     province matches
6064    ceuta        
6065  melilla        
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['ciudad', 'prov_co', 'prov_no2', 'prov_o3', 'prov_pm10', 'prov_pm25',
       'prov_so2', 'province', 'matches'],
      dtype='object')
num registros tras union: (52, 7)


In [1430]:
union_agrup_mar_5.head(2)

,prov_co,prov_no2,prov_o3,prov_pm10,prov_pm25,prov_so2,province
0,349.716,15.0428,51.0958,44.9275,NaN,3.02055,navarra
1,275.38,25.9469,44.1475,43.7906,NaN,1.73006,barcelona


In [1431]:
union_mar_5 = pd.merge(union_mar_4, union_agrup_mar_5, left_on='province',right_on='province', how='left')

In [1432]:
# comprobacion duplicados
union_mar_5[union_mar_5.duplicated()]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual,prov_co,prov_no2,prov_o3,prov_pm10,prov_pm25,prov_so2


In [1433]:
union_mar_5.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual,prov_co,prov_no2,prov_o3,prov_pm10,prov_pm25,prov_so2
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,124813.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.641,11.2318,60.0819,57.1706,17.9722,1.87143
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,8506.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.641,11.2318,60.0819,57.1706,17.9722,1.87143


In [1434]:
union_mar_5.shape

(6066, 88)

In [1435]:
union_mar_5.to_csv("union_mar2.csv")

### Ficheros Guille

##### Trafico

In [1436]:
trafico = pd.read_csv('../Guille/recursos_trafico_carreteras_clean_2018.csv',index_col=0)
trafico.shape

(71, 4)

In [1437]:
trafico.head(2)

,CCAA_name,Provincia_name,Total: Millones vehículos-km,porcentaje_carreteras_ccaa_y_estado
0,andalucia,almeria,3425.81,0.830417
1,andalucia,cadiz,4725.45,0.877934


In [1438]:
trafico['Provincia_name'].replace("palmas_las","las_palmas", inplace=True, regex=True)
trafico.rename(columns={'Total: Millones vehículos-km': 'total_millones_vehiculos_km'}, inplace=True)
trafico.drop(columns=['CCAA_name'], inplace=True)
trafico.drop_duplicates(subset=['Provincia_name'], inplace=True)

In [1439]:
new_names = [(i,'prov_'+i) for i in trafico.iloc[:, 1:].columns.values]
trafico.rename(columns = dict(new_names), inplace=True)

In [1440]:
union_agrup_guille_1 = union_agrup(maestro_nivel_municipio_ori, trafico, 'province_es', 'Provincia_name', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 66
num agrupaciones maestro: (52, 1)
num agrupaciones union: (66, 1)
num agrupaciones tras match: (52, 2)
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['Provincia_name', 'prov_total_millones_vehiculos_km',
       'prov_porcentaje_carreteras_ccaa_y_estado', 'province_es', 'matches'],
      dtype='object')
num registros tras union: (52, 3)


In [1441]:
union_agrup_guille_1.head(2)

,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado,province_es
0,3425.81,0.830417,almeria
1,4725.45,0.877934,cadiz


In [1442]:
union_guille_1 = pd.merge(maestro_nivel_municipio_ori, union_agrup_guille_1, left_on='province_es',right_on='province_es', how='left')

In [1443]:
# comprobacion duplicados
union_agrup_guille_1[union_agrup_guille_1.duplicated()]

,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado,province_es


In [1444]:
union_guille_1.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,1869.3,0.832942
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,1869.3,0.832942


In [1445]:
union_guille_1.shape

(6066, 12)

##### Empresas_autobuses

In [1446]:
empresas_autobuses = pd.read_csv('../Guille/recursos_empresas_autobuses_clean.csv',index_col=0)
empresas_autobuses.shape

(77, 2)

In [1447]:
empresas_autobuses.head(2)

,Municipio,bus_based_companies_count
0,barcelona,5
1,caceres,3


In [1448]:
empresas_autobuses['municipio_limpio'] = empresas_autobuses['Municipio'].apply(doClean)
empresas_autobuses.drop(columns='Municipio', inplace=True)

In [1449]:
empresas_autobuses.head()

,bus_based_companies_count,municipio_limpio
0,5,barcelona
1,3,caceres
2,3,jaen
3,3,soria
4,3,salamanca


In [1450]:
union_guille_2 = union_solo_muni(union_guille_1, empresas_autobuses, 'cities_es', 'municipio_limpio', threshold=93)

num registros maestro: 6066
num registros fichero para unir: 77
num agrupaciones tras match: (6066, 13)
no se han cruzado todos los valores:
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['bus_based_companies_count', 'municipio_limpio', 'cities_es', 'cities',
       'province_es', 'province', 'ccaa_es', 'ccaa', 'cities_es/province',
       'provincia_original', 'municipio_original', 'muni_agrup',
       'prov_total_millones_vehiculos_km',
       'prov_porcentaje_carreteras_ccaa_y_estado', 'matches'],
      dtype='object')
num registros tras union: (6066, 13)


In [1451]:
union_guille_2[~union_guille_2['bus_based_companies_count'].isnull()].shape

(65, 13)

##### Cobertura red

In [1452]:
cobertura_media = pd.read_csv('../Guille/recursos_cobertura_media_clean_2018.csv',index_col=0)
cobertura_media.shape

(8124, 4)

In [1453]:
cobertura_media.head(2)

,Provinicia,Municipio,Cod ESP,Calidad_Cobertura_fija&movil_%
0,araba,alegria_dulantzi,1001000100,65.00
1,araba,amurrio,1002000100,62.75


In [1454]:
cobertura_media['Provinicia'].replace("balears_illes","illes_balears", inplace=True, regex=True)
cobertura_media['Provinicia'].replace("palmas_las","las_palmas", inplace=True, regex=True)
cobertura_media['Provinicia'].replace("rioja_la","la_rioja", inplace=True, regex=True)

In [1455]:
cobertura_media.rename(columns={'Calidad_Cobertura_fija&movil_%': 'perc_calidad_cobertura_fija_movil'}, inplace=True)
cobertura_media.drop(columns=['Cod ESP'], inplace=True)
cobertura_media.drop_duplicates(inplace=True)

In [1456]:
union_guille_3 = union_muni(union_guille_2, cobertura_media, 'cities_es', 'province', 'Municipio', 'Provinicia', threshold=80, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 8124
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
df_union_agrup_match: (8124, 5)
barcelona
no se han cruzado todos los valores:
caceres
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
soria
no se han cruzado todos los valores:
salamanca
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
sevilla
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
bizkaia
coruna
no se han cruzado todos los valores:
castellon
no se han cruzado todos los valores:
huelva
no se han cruzado todos los valores:
madrid
no se han cruzado todos los valores:
valencia
no se han cruzado todos los valores:
malaga
no se han cruzado todos los valores:
ourense
no se han cruzado todos los valores:
almeria
no se han cruzado todos los valores:
badajoz
no se han cruza

In [1457]:
union_guille_3[~union_guille_3['perc_calidad_cobertura_fija_movil'].isnull()].shape

(5998, 13)

In [1458]:
union_guille_3.shape

(6066, 13)

##### Accesibilidad

In [1459]:
accesibilidad_media_ccaa = pd.read_csv('../Guille/recursos_accesibilidad_ciudades_ccaa_2015.csv',index_col=0)
accesibilidad_media_ccaa.shape

(18, 5)

In [1460]:
accesibilidad_media_ccaa.head(2)

,GID_0,NAME_0,GID_1,NAME_1,accessibility_mean
853,ESP,Spain,ESP.1_1,andalucia,51.600938
854,ESP,Spain,ESP.2_1,aragon,62.648308


In [1461]:
accesibilidad_media_ccaa.rename(columns={'NAME_1': 'ccaa_limpio'}, inplace=True)
accesibilidad_media_ccaa.drop(columns=['GID_0', 'NAME_0', 'GID_1'], inplace=True)
accesibilidad_media_ccaa.drop_duplicates(subset=['ccaa_limpio'], inplace=True)

In [1462]:
accesibilidad_media_ccaa['ccaa_limpio'].replace("islas_canarias", "canarias", inplace=True, regex=True)

In [1463]:
new_names = [(i,'ccaa_'+i) for i in accesibilidad_media_ccaa.iloc[:, :].columns.values]
accesibilidad_media_ccaa.rename(columns = dict(new_names), inplace=True)

In [1464]:
union_agrup_guille_4 = union_agrup(union_guille_3, accesibilidad_media_ccaa, 'ccaa_es', 'ccaa_ccaa_limpio', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 18
num agrupaciones maestro: (19, 1)
num agrupaciones union: (18, 1)
num agrupaciones tras match: (19, 2)
no se han cruzado todos los valores:
Empty DataFrame
Columns: [ccaa_es, matches]
Index: []
el fichero de union no dispone de todos los valores para cruzar
columnas final: Index(['ccaa_ccaa_limpio', 'ccaa_accessibility_mean', 'ccaa_es', 'matches'], dtype='object')
num registros tras union: (19, 2)


In [1465]:
union_agrup_guille_4.head(2)

,ccaa_accessibility_mean,ccaa_es
0,51.600938,andalucia
1,62.648308,aragon


In [1466]:
union_guille_4 = pd.merge(union_guille_3, union_agrup_guille_4, left_on='ccaa_es',right_on='ccaa_es', how='left')

In [1467]:
# comprobacion duplicados
union_guille_4[union_guille_4.duplicated()]

,bus_based_companies_count,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado,perc_calidad_cobertura_fija_movil,ccaa_accessibility_mean


In [1468]:
union_guille_4.head(2)

,bus_based_companies_count,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado,perc_calidad_cobertura_fija_movil,ccaa_accessibility_mean
0,5.0,barcelona,barcelona,barcelona,barcelona,cataluna,cataluna,barcelona/barcelona,Barcelona,Barcelona,8838.35,0.844502,100.0,37.202517
1,1.0,terrassa,terrassa,barcelona,barcelona,cataluna,cataluna,terrassa/barcelona,Barcelona,Terrassa,8838.35,0.844502,92.0,37.202517


In [1469]:
union_guille_4.shape

(6066, 14)

##### Tren

In [1470]:
estaciones = pd.read_csv('../Guille/estaciones_renfe_prov.csv',index_col=None)
estaciones.shape

(1498, 8)

In [1471]:
estaciones.head(2)

,CÓDIGO,C.P.,POBLACION,PROVINCIA,CERCANIAS,FEVE,AVE,numero_estaciones
0,31412,15008,coruna_a,coruna_a,NO,NO,SI,2
1,5129,15569,cerdido,coruna_a,NO,SI,NO,2


In [1472]:
estaciones[estaciones['POBLACION']=='benicasim']
# idioma municipio = es

,CÓDIGO,C.P.,POBLACION,PROVINCIA,CERCANIAS,FEVE,AVE,numero_estaciones
186,65318,12560,benicasim,castellon,NO,NO,NO,1


In [1473]:
estaciones['PROVINCIA'].replace("rioja_la","la_rioja", inplace=True, regex=True)
estaciones.drop(columns=['C.P.', 'CÓDIGO'], inplace=True)
estaciones = estaciones.groupby(by=['POBLACION','PROVINCIA'], as_index=False).agg(lambda x : x.sum() if x.dtype=='int64' else x.max())
estaciones.shape

(899, 6)

In [1474]:
estaciones['CERCANIAS'] = estaciones['CERCANIAS'].map({'SI': 1, 'NO': 0})
estaciones['FEVE'] = estaciones['FEVE'].map({'SI': 1, 'NO': 0})
estaciones['AVE'] = estaciones['AVE'].map({'SI': 1, 'NO': 0})
estaciones.rename(columns={'CERCANIAS': 'tiene_cercanias',
                          'FEVE': 'tiene_feve',
                          'AVE': 'tiene_ave',
                          'numero_estaciones': 'nume_estaciones_tren'}, inplace=True)

In [1475]:
union_guille_5 = union_muni(union_guille_4, estaciones, 'cities_es', 'province', 'POBLACION', 'PROVINCIA', threshold=85, threshold_mun=90)

num registros maestro: 6066
num registros fichero para unir: 899
num agrupaciones maestro: (52, 1)
num agrupaciones union: (47, 1)
num agrupaciones tras match: (52, 2)
no se han cruzado todos los valores:
                    province matches
3805           illes_balears        
5114              las_palmas        
5149  santa_cruz_de_tenerife        
6064                   ceuta        
6065                 melilla        
el fichero de union no dispone de todos los valores para cruzar
df_union_agrup_match: (904, 8)
barcelona
no se han cruzado todos los valores:
caceres
no se han cruzado todos los valores:
jaen
no se han cruzado todos los valores:
soria
no se han cruzado todos los valores:
salamanca
no se han cruzado todos los valores:
granada
no se han cruzado todos los valores:
sevilla
no se han cruzado todos los valores:
ciudad_real
no se han cruzado todos los valores:
alicante
no se han cruzado todos los valores:
bizkaia
no se han cruzado todos los valores:
coruna
no se han cruzado

In [1476]:
union_guille_5[union_guille_5['nume_estaciones_tren']>0].shape

(3717, 18)

In [1477]:
union_guille_5.shape

(6066, 18)

In [1478]:
union_guille_5.to_csv("union_guille2.csv")

### Ficheros Sandra

In [1481]:
comercios = pd.read_csv("../Sandra/Economia_comercios_municipios_2019.csv", dtype={'CP': object})
comercios.shape

(8189, 13)

In [1482]:
comercios.head(5)

,CP,Municipio,Total Comercios,B_E Industria,F Construcción,"Comercio, transporte y hostelería",J Información y comunicaciones,K Actividades financieras y de seguros,L Actividades inmobiliarias,Actividades profesionales y técnicas,"Educación, sanidad y servicios sociales",Otros servicios personales,Total servicios
0,0,Total Nacional,"3,363,197","199,076","422,350","1,215,680","68,158","77,396","179,484","622,015","272,124","306,914","1,526,091"
1,01,Araba/Álava,"18,678","1,942","2,397","7,099",359,326,221,"3,132","1,493","1,709","7,240"
2,01001,Alegría-Dulantzi,148,20,26,59,,,,,,,43
3,01002,Amurrio,600,63,95,251,6,9,7,90,40,39,191
4,01003,Aramaio,50,8,7,16,,,,,,,19


In [1483]:
comercios = comercios.rename(columns=standardize_headers)
comercios.drop(columns='municipio', inplace=True)
comercios['cp'] = comercios['cp'].str.strip(' ')

In [1484]:
comercios.columns

Index(['cp', 'total_comercios', 'b_e_industria', 'f_construccion',
       'comercio_transporte_y_hosteleria', 'j_informacion_y_comunicaciones',
       'k_actividades_financieras_y_de_seguros', 'l_actividades_inmobiliarias',
       'actividades_profesionales_y_tecnicas',
       'educacion_sanidad_y_servicios_sociales', 'otros_servicios_personales',
       'total_servicios'],
      dtype='object')

In [1485]:
for column in ['total_comercios', 'b_e_industria', 'f_construccion',
       'comercio_transporte_y_hosteleria', 'j_informacion_y_comunicaciones',
       'k_actividades_financieras_y_de_seguros', 'l_actividades_inmobiliarias',
       'actividades_profesionales_y_tecnicas',
       'educacion_sanidad_y_servicios_sociales', 'otros_servicios_personales',
       'total_servicios']:
        comercios[column] = comercios[column].apply(doCleanNumber)

#comercios['b_e_industria'] = comercios['b_e_industria'].apply(doCleanNumber)

In [1486]:
comercios.head()

,cp,total_comercios,b_e_industria,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios
0,0,3363197.0,199076.0,422350.0,1215680.0,68158.0,77396.0,179484.0,622015.0,272124.0,306914.0,1526091.0
1,01,18678.0,1942.0,2397.0,7099.0,359.0,326.0,221.0,3132.0,1493.0,1709.0,7240.0
2,01001,148.0,20.0,26.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0
3,01002,600.0,63.0,95.0,251.0,6.0,9.0,7.0,90.0,40.0,39.0,191.0
4,01003,50.0,8.0,7.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0


In [1487]:
#union_sandra_1 = union_solo_muni(maestro_nivel_municipio_ori, comercios, 'cities_es', 'municipio', threshold=90)

In [1488]:
union_sandra_1 = union_cod_postal(maestro_nivel_cod_postal, comercios, maestro_nivel_municipio_ori, 'postal_code', 'cp', 'cities_es/province', 'sum')
union_sandra_1.shape

(6066, 21)

In [1489]:
union_sandra_1[union_sandra_1['total_comercios']>0].shape

(970, 21)

In [1490]:
union_sandra_1.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,b_e_industria,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,aramaio/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,artziniega/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,arminon/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### IRPF / Salario - DESCARTADO

In [1491]:
irpf = pd.read_csv("../Sandra/Economia_IRPF_municipios_2017.csv", index_col=0)
irpf.shape

(2897, 6)

In [1492]:
irpf.head(2)

,n_habitantes,Renta_disponible,CCAA,PROVINCIA,MUNICIPIO,CP_AT
57,1005.0,8092773.17,aragon,zaragoza,bujaraloz,50059.0
58,1047.0,8434679.97,comunitat_valenciana,valencia,palmera,46188.0


In [1493]:
irpf['CCAA'].drop_duplicates()

57                     aragon
58       comunitat_valenciana
59                   cataluna
60             castillay_leon
61                  andalucia
251        castilla_la_mancha
472                 cantabria
517                  canarias
560                   galicia
600     principadode_asturias
684           regionde_murcia
688                       NaN
736        comunidadde_madrid
837                  la_rioja
1367              extremadura
1531            illes_balears
2816                  melilla
2817                    ceuta
Name: CCAA, dtype: object

In [1494]:
irpf.drop(columns=['CCAA', 'CP_AT', 'n_habitantes'], inplace=True)
irpf.rename(columns={'Renta_disponible': 'renta_disponible'}, inplace=True)
irpf = irpf[~irpf['MUNICIPIO'].isnull()]

In [1495]:
irpf[irpf[['PROVINCIA', 'MUNICIPIO']].duplicated()]

,renta_disponible,PROVINCIA,MUNICIPIO


In [1496]:
# faltan 4 provincias
#union_sandra_2 = union_muni(union_sandra_1, irpf, 'cities_es', 'province_es', 'MUNICIPIO', 'PROVINCIA', threshold=80, threshold_mun=90)

##### PIB

In [1567]:
pib = pd.read_csv("../Sandra/Economia_PIB_provincias_2017.csv", index_col=0 )
pib.shape

(52, 2)

In [1568]:
pib.head(2)

,provincia,PIB pm
0,albacete,7762548
1,alicante,35061866


In [1569]:
pib['provincia'].drop_duplicates().head(2)
# faltan 4 provincias

0    albacete
1    alicante
Name: provincia, dtype: object

In [1570]:
pib['provincia'].replace("balears_illes","illes_balears", inplace=True, regex=True)
pib['provincia'].replace("palmas_las","las_palmas", inplace=True, regex=True)
pib['provincia'].replace("rioja_la","la_rioja", inplace=True, regex=True)

In [1571]:
pib.rename(columns={'PIB pm': 'pib'}, inplace=True)

new_names = [(i,'prov_'+i) for i in pib.iloc[:, :].columns.values]
pib.rename(columns = dict(new_names), inplace=True)

In [1572]:
union_agup_sandra_3 = union_agrup(maestro_nivel_municipio_ori, pib, 'province', 'prov_provincia', threshold=85)

num registros maestro: 6066
num registros fichero para unir: 52
num agrupaciones maestro: (52, 1)
num agrupaciones union: (52, 1)
num agrupaciones tras match: (52, 2)
columnas final: Index(['prov_provincia', 'prov_pib', 'province', 'matches'], dtype='object')
num registros tras union: (52, 2)


In [1573]:
union_agup_sandra_3.head(2)

,prov_pib,province
0,7762548,albacete
1,35061866,alicante


In [1574]:
union_sandra_3 = pd.merge(union_sandra_1, union_agup_sandra_3, left_on='province',right_on='province', how='left')

In [1575]:
# comprobacion duplicados
union_sandra_3[union_sandra_3.duplicated()]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios,prov_pib


In [1576]:
union_sandra_3.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios,prov_pib
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377


In [1577]:
union_sandra_3.shape

(6066, 22)

In [1578]:
union_sandra_3.to_csv("union_sandra2.csv")

## Bloques

In [4]:
maestro = pd.read_csv("maestro/maestro_nivel_municipio_ori.csv", index_col=0 )
maestro.shape

(6066, 9)

In [5]:
maestro[maestro.duplicated(subset=['cities_es/province'])]

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original


In [6]:
maestro[maestro['cities_es/province']=='alicante/alicante']

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original
129,alicante,alicante,alicante,alicante,comunidad_valenciana,comunitat_valenciana,alicante/alicante,Alicante/Alacant,Alicante/Alacant


In [7]:
maestro.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón


In [8]:
patri = pd.read_csv("maestro/union_patri2.csv", index_col=0 )
patri.shape

(6066, 51)

In [9]:
patri.head(2)

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,...,num_establec_otros_servicios,num_establec_palacios_de_congresos,num_establec_playas,num_establec_puertos_deportivos,num_establec_restaurantes_y_empresas_turísticas_de_catering,num_establec_seguridad_ciudadana,num_establec_servicios_de_limpieza,num_establec_servicios_turísticos_de_salud,num_establec_taxis,num_establec_turismo_industrial
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
patri.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province', 'provincia_original', 'municipio_original',
       'num_bibliotecas', 'num_hospitales', 'num_centros_salud',
       'num_colegios_privados', 'num_colegios_publicos', 'num_guardia_civil',
       'num_universidades_privadas', 'num_universidades_publicas',
       'num_tit_uni_privadas', 'num_tit_uni_publicas',
       'num_establec_agencias_de_viajes', 'num_establec_albergues',
       'num_establec_alojamientos_rurales', 'num_establec_artesanos',
       'num_establec_atraque_de_cruceros_y_ferries',
       'num_establec_bares_y_cafeterías', 'num_establec_bodegas',
       'num_establec_campings',
       'num_establec_campos_de_golf_y_campos_de_pitch&putt',
       'num_establec_comercios', 'num_establec_convention_bureaux',
       'num_establec_empresas_de_alquiler_de_vehículos_y_embarcaciones',
       'num_establec_empresas_de_transporte_turístico',
       'num_establec_empresas_de_turism

In [11]:
patri.drop(columns=['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'provincia_original', 'municipio_original'], inplace=True)

In [12]:
union = pd.merge(maestro, patri, left_on='cities_es/province', right_on='cities_es/province', how='left')
union.shape

(6066, 51)

In [13]:
mar = pd.read_csv("maestro/union_mar2.csv", index_col=0 )
mar.shape

(6066, 88)

In [14]:
mar.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,hombres_total,...,prov_nw_91_anual,prov_w_med_anual,prov_ta_max_anual,prov_ta_min_anual,prov_co,prov_no2,prov_o3,prov_pm10,prov_pm25,prov_so2
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,124813.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.640625,11.231788,60.081871,57.170569,17.972222,1.871429
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,8506.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.640625,11.231788,60.081871,57.170569,17.972222,1.871429
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,764.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.640625,11.231788,60.081871,57.170569,17.972222,1.871429
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,2406.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.640625,11.231788,60.081871,57.170569,17.972222,1.871429
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,2523.0,...,0.25,12.545455,39.7(29/jun),-7.7(06/ene),181.640625,11.231788,60.081871,57.170569,17.972222,1.871429


In [15]:
mar.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province', 'provincia_original', 'municipio_original',
       'hombres_total', 'hombres_de_0_4', 'hombres_de_5_9', 'hombres_de_10_4',
       'hombres_de_15_19', 'hombres_de_20_24', 'hombres_de_25_29',
       'hombres_de_30_34', 'hombres_de_35_39', 'hombres_de_40_44',
       'hombres_de_45_49', 'hombres_de_50_54', 'hombres_de_55_59',
       'hombres_de_60_64', 'hombres_de_65_69', 'hombres_de_70_74',
       'hombres_de_75_79', 'hombres_de_80_84', 'hombres_de_85_89',
       'hombres_de_90_94', 'hombres_de_95_99', 'hombres_de_100_y_mas',
       'mujeres_total', 'mujeres_de_0_4', 'mujeres_de_5_9', 'mujeres_de_10_4',
       'mujeres_de_15_19', 'mujeres_de_20_24', 'mujeres_de_25_29',
       'mujeres_de_30_34', 'mujeres_de_35_39', 'mujeres_de_40_44',
       'mujeres_de_45_49', 'mujeres_de_50_54', 'mujeres_de_55_59',
       'mujeres_de_60_64', 'mujeres_de_65_69', 'mujeres_de_70_74',
       'mujeres_de_

In [16]:
mar.drop(columns=['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'provincia_original', 'municipio_original'], inplace=True)

In [17]:
union = pd.merge(union, mar, left_on='cities_es/province', right_on='cities_es/province', how='left')
union.shape

(6066, 130)

In [18]:
guille = pd.read_csv("maestro/union_guille2.csv", index_col=0 )
guille.shape

(6066, 18)

In [19]:
guille.head()

,bus_based_companies_count,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,prov_total_millones_vehiculos_km,prov_porcentaje_carreteras_ccaa_y_estado,perc_calidad_cobertura_fija_movil,ccaa_accessibility_mean,tiene_cercanias,tiene_feve,tiene_ave,nume_estaciones_tren
0,5.0,barcelona,barcelona,barcelona,barcelona,cataluna,cataluna,barcelona/barcelona,Barcelona,Barcelona,8838.35,0.844502,100.00,37.202517,1.0,0.0,1.0,100.0
1,1.0,terrassa,terrassa,barcelona,barcelona,cataluna,cataluna,terrassa/barcelona,Barcelona,Terrassa,8838.35,0.844502,92.00,37.202517,1.0,0.0,0.0,4.0
2,1.0,sant_boi_de_llobregat,sant_boi_de_llobregat,barcelona,barcelona,cataluna,cataluna,sant_boi_de_llobregat/barcelona,Barcelona,Sant Boi de Llobregat,8838.35,0.844502,100.00,37.202517,1.0,0.0,0.0,1.0
3,NaN,abrera,abrera,barcelona,barcelona,cataluna,cataluna,abrera/barcelona,Barcelona,Abrera,8838.35,0.844502,100.00,37.202517,1.0,0.0,1.0,100.0
4,NaN,aguilar_de_segarra,aguilar_de_segarra,barcelona,barcelona,cataluna,cataluna,aguilar_de_segarra/barcelona,Barcelona,Aguilar de Segarra,8838.35,0.844502,28.33,37.202517,0.0,0.0,0.0,1.0


In [20]:
guille.columns

Index(['bus_based_companies_count', 'cities_es', 'cities', 'province_es',
       'province', 'ccaa_es', 'ccaa', 'cities_es/province',
       'provincia_original', 'municipio_original',
       'prov_total_millones_vehiculos_km',
       'prov_porcentaje_carreteras_ccaa_y_estado',
       'perc_calidad_cobertura_fija_movil', 'ccaa_accessibility_mean',
       'tiene_cercanias', 'tiene_feve', 'tiene_ave', 'nume_estaciones_tren'],
      dtype='object')

In [21]:
guille.drop(columns=['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'provincia_original', 'municipio_original'], inplace=True)

In [22]:
union = pd.merge(union, guille, left_on='cities_es/province', right_on='cities_es/province', how='left')
union.shape

(6066, 139)

In [23]:
sandra = pd.read_csv("maestro/union_sandra2.csv", index_col=0 )
sandra.shape

(6066, 22)

In [24]:
sandra.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,muni_agrup,...,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios,prov_pib
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,alegria_dulantzi/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,amurrio/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,aramaio/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,artziniega/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,arminon/pais_vasco,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377


In [25]:
sandra.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province', 'provincia_original', 'municipio_original',
       'muni_agrup', 'total_comercios', 'b_e_industria', 'f_construccion',
       'comercio_transporte_y_hosteleria', 'j_informacion_y_comunicaciones',
       'k_actividades_financieras_y_de_seguros', 'l_actividades_inmobiliarias',
       'actividades_profesionales_y_tecnicas',
       'educacion_sanidad_y_servicios_sociales', 'otros_servicios_personales',
       'total_servicios', 'prov_pib'],
      dtype='object')

In [26]:
sandra.drop(columns=['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'provincia_original', 'municipio_original', 'muni_agrup'], inplace=True)

In [27]:
union = pd.merge(union, sandra, left_on='cities_es/province', right_on='cities_es/province', how='left')
union.shape

(6066, 151)

In [28]:
union.fillna(-1, inplace=True)

In [29]:
union.head()

,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province,provincia_original,municipio_original,num_bibliotecas,...,f_construccion,comercio_transporte_y_hosteleria,j_informacion_y_comunicaciones,k_actividades_financieras_y_de_seguros,l_actividades_inmobiliarias,actividades_profesionales_y_tecnicas,educacion_sanidad_y_servicios_sociales,otros_servicios_personales,total_servicios,prov_pib
0,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/araba,Araba/Álava,Alegría-Dulantzi,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
1,amurrio,amurrio,alava,araba,pais_vasco,pais_vasco,amurrio/araba,Araba/Álava,Amurrio,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
2,aramaio,aramaio,alava,araba,pais_vasco,pais_vasco,aramaio/araba,Araba/Álava,Aramaio,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
3,artziniega,artziniega,alava,araba,pais_vasco,pais_vasco,artziniega/araba,Araba/Álava,Artziniega,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377
4,arminon,arminon,alava,araba,pais_vasco,pais_vasco,arminon/araba,Araba/Álava,Armiñón,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11983377


## Exportar fichero union

In [30]:
union.to_csv("union_todo.csv")